# Raster Factories Tutorial
This tutorial shows how to use the `Raster` class to load and build raster datasets from different data sources.

## Introduction

There are many ways to acquire raster datasets when running hazard assessments. For example, a dataset may be loaded from a local file or web URL, or alternatively produced by performing computations on a numpy array. In other cases, you may want to rasterize a non-raster dataset - for example, converting a collection of Polygon features to a Raster representation.

To accommodate these diverse cases, the `Raster` class provides factory methods that create `Raster` objects for different data sources. Factories are the recommended way to create `Raster` objects, and they follow the naming convention `from_<source>`, where `<source>` indicates a particular type of data source. Commonly used factories include:

* `from_file`: Loads a raster from the local filesystem
* `from_url`: Loads a raster from a web URL
* `from_array`: Builds a `Raster` from a numpy array
* `from_points`: Builds a raster from a collection of Point features
* `from_polygons`: Builds a raster from a collection of Polygon features

and this tutorial will examine each of these commands.

Finally, it is also often useful to save `Raster` datasets to the local filesystem. For example, to save a new raster produced by a hazard assessment analysis. The `Raster` class provides the `save` command for just this purpose, and we will examine that command at the end of the tutorial.

## Prerequisites

### Install pfdf
To run this tutorial, you must have installed [pfdf 3+ with tutorial resources](https://ghsc.code-pages.usgs.gov/lhp/pfdf/resources/installation.html#tutorials) in your Jupyter kernel. The following line checks this is the case:

In [ ]:
import check_installation

### Imports
We'll next import the ``Raster`` class from pfdf. We'll also use `numpy` to create some example datasets, `pathlib.Path` to work with saved files, and the `plot` module to visualize datasets.

In [ ]:
from pfdf.raster import Raster
import numpy as np
from pathlib import Path
from tools import plot

### Example Files
Finally, we'll save some example files to use in the tutorial. The raster dataset is a 50x75 grid of random values between 0 and 100 with a border of -1 NoData values along the edges. The raster is projected in EPSG:26911 with a 10 meter resolution. We'll also save a raster mask that locates pixels with a value greater than 50, and an example Polygon feature collection.

In [ ]:
from tools import examples
examples.build_raster()
examples.build_mask()
examples.build_polygons()

## Raster.from_file
You can use the `Raster.from_file` command to return a `Raster` object for a dataset saved to the local filesystem. For example:

In [ ]:
Raster.from_file('examples/raster.tif')

Inspecting our example dataset, we can see the `Raster` holds the full 50x75 pixel data grid, which spans from 0 to 750 across the X axis, and from -500 to 0 along the Y axis.

The `from_file` command includes a variety of options, which you can read about in the [API](https://ghsc.code-pages.usgs.gov/lhp/pfdf/api/raster.html#pfdf.raster.Raster.from_file). One commonly used option is the `bounds` input, which lets you specify a bounding box in which to read data. This can be useful when your area-of-interest is much smaller than the saved raster dataset, and you want to limit the amount of data read into memory.

For example, let's try loading only the top-left quadrant of our example raster:

In [ ]:
bounds = {'left': 0, 'bottom': -250, 'right': 370, 'top': 0, 'crs': 26911}
Raster.from_file('examples/raster.tif', bounds=bounds)

Inspecting the `Raster`, we can see that commandonly loaded data for the 25x37 pixel grid in the top-left corner of the saved dataset. The bounding box is correspondingly smaller, and spans from 0 to 370 along the X axis, and -250 to 0 along the Y axis.

Note that the `bounds` input does not need to use the same CRS as the saved file. For example, here we'll load the same subset of the example file, but using a bounding box defined in EPSG:4326 instead:

In [ ]:
bounds = [
    -121.48874388784702,    # left
    -0.0022548542591846253, # bottom
    -121.4854290480384,     # right
    0.0,                    # top
    4326,                   # crs
]
Raster.from_file('examples/raster.tif', bounds=bounds)

## Raster.from_url
You can use `Raster.from_url` to load a raster from a web URL. The command supports all the options of `Raster.from_file`, with some additional options for establishing web connections. We recommend using the `bounds` options with `Raster.from_url` whenever possible. Just like the `from_file` command, the `bounds` option instructs the command to only load data from the indicated bounding. This helps limit the total amount of data that needs to be transferred over an internet connection.

For example, the USGS distributes its 10 meter DEM product as a tiled dataset, with each tile spanning 1x1 degree of longitude and latitude. The tiles can be accessed via web URLs, but each tile requires ~400 MB of memory, which can take a while to download. Here, we'll use the `bounds` option to download a subset of data from one of these tiles. Specifically, we'll download DEM data near the town of Golden, Colorado:

In [ ]:
bounds = {
    'left': -105.239773,
    'bottom': 39.739556,
    'right': -105.206539,
    'top': 39.782944,
    'crs': 4326,
}
url = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/historical/n40w106/USGS_13_n40w106_20230602.tif'
raster = Raster.from_url(url, bounds=bounds)
print(raster)

Inspecting the memory footprint of the loaded data, we can see it uses ~673 KB, significantly less than the ~400 MB of the full DEM tile:

In [ ]:
print(f'nbytes = {raster.nbytes}')

One of the key connection options is the `timeout` parameter. This option specifies a maximum time to establish an initial connection with the URL server. (This is **not** the total download time, which can be much longer). By default, this is set to 10 seconds, but you might consider raising this time limit if you're on a slow connection. For example:

In [ ]:
# Allows a full minute to establish a connection
raster = Raster.from_url(url, bounds=bounds, timeout=60)

## Boolean Masks
Boolean raster masks are commonly used when working with pfdf. A raster mask is a raster dataset in which all the data values are 0/False or 1/True. These values are used to selectively choose pixels in an associated data raster of the same shape. For example, you might use a mask to indicate pixels that should be included/ignored in an analysis.

When working with numpy arrays, raster masks are best represented as arrays with a ``bool`` (boolean) dtype. However, many raster file formats do not support boolean data types, so instead save masks as the integers 1 and 0. This can cause problems, as numpy interprets integer indices differently from booleans.

As such, the `from_file` and `from_url` factories include a `isbool` option. Setting this option to True indicates that a saved raster represents a boolean mask, rather than an integer data array. When you use this option, the `Raster` object's data array will have a ``bool`` dtype, and will be suitable for masking with numpy. We **strongly recomend** using this option whenever you load a saved raster mask.

For example, if we naively load our example mask, we can see the output `Raster` has an integer dtype:

In [ ]:
raster = Raster.from_file('examples/mask.tif')
print(raster.dtype)
print(raster.values)

But if we use the `isbool` option, then the `Raster` has the correct dtype and can be used for numpy indexing:

In [ ]:
raster = Raster.from_file('examples/mask.tif', isbool=True)
print(raster.dtype)
print(raster.values)

## Raster.from_array
You can use the `from_array` factory to build a `Raster` from a numpy array or array-like object.

### Spatial Metadata
Since numpy arrays do not include spatial metadata, a basic call to this factory will result in a `Raster` object without spatial metadata. For example:

In [ ]:
values = np.arange(100).reshape(10,10)
Raster.from_array(values)

As such, the `from_array` factory includes a variety of options to specify this metadata. For example, you can use the `crs` option to specify a CRS, and either the `transform` or `bounds` option to specify either an affine transform or a bounding box:

In [ ]:
Raster.from_array(values, crs=4326, bounds=[0,0,50,200])

Note that you can only provide one of the `transform` or `bounds` options, as they actually provide the same information, albeit in different formats.

Alternatively, you can use the `spatial` option to set the spatial metadata equal to the spatial metadata of another `Raster` object. This can be useful when performing computations on a raster's data grid. For example, lets do some math on our example raster's data grid, and then convert the results to a `Raster` object:

In [ ]:
template = Raster.from_file('examples/raster.tif')
results = template.values * 1.2
Raster.from_array(results, spatial=template)

Inspecting the output, we can see the new `Raster` object has the same CRS, transform, and bounding box as the template raster.

### NoData Value


By default, `from_array` will attempt to determine a NoData value for the `Raster` from the array dtype. As follows:

* float: nan
* signed integers: most negative representable value
* unsigned integers: most postive representable value

Alternatively, you can use the `nodata` option to specify the NoData value explicitly. For example:

In [ ]:
values = np.zeros((10,10), float)
default = Raster.from_array(values)
explicit = Raster.from_array(values, nodata=-1)

print(default.nodata)
print(explicit.nodata)

## Raster.from_polygons

You can also use raster factories to rasterize vector feature datasets. We'll begin with the `from_polygons` factory, which converts a Polygon/MultiPolygon feature collection to a raster. The command requires the path to a vector feature file as input. For example, the `perimeter.geojson` data file is a Polygon collection:

In [ ]:
raster = Raster.from_polygons('examples/polygons.geojson')
print(raster)

### Resolution

By default, the command rasterizes polygons to a 10 meter resolution, which is the recommended resolution for many of pfdf's hazard models:

In [ ]:
raster.resolution('meters')

But you can use the `resolution` option to set a different resolution instead:

In [ ]:
raster = Raster.from_polygons('examples/polygons.geojson', resolution=30, units='meters')
raster.resolution('meters')

### Data Fields

By default, `from_polygons` will build a boolean raster mask, where True pixels indicate locations within one of the polygons. However, you can use the `field` option to build the raster from one of the polygon data fields instead. In this case, pixels in a polygon will be set to the value for that polygon, and all other pixels are NoData. 

For example, the example polygon dataset consists of five squares. If we don't specify a data field, then the output raster masks the pixels in those squares:

In [ ]:
raster = Raster.from_polygons('examples/polygons.geojson')
plot.mask(raster, title='Rasterized Polygons (Boolean Mask)', basemap=False)

By contrast, if we rasterize the polygons from a data field, then the pixel values are determined by the data field:

In [ ]:
raster = Raster.from_polygons('examples/polygons.geojson', field='my-data')
plot.raster(raster, title='Rasterized Polygons (Data Field)', cmap='OrRd', clabel='Data Field')

### Bounding Box
It's not uncommon to have a Polygon dataset that covers a much larger spatial area than your area of interest. When this is the case, rasterizing the entire polygon dataset can require an excessive amount of memory. Instead, use the `bounds` option to only rasterize the polygon features that intersect the given bounding box.

For example, let's use the `bounds` option to only rasterize the bottom-left quadrant of our example dataset. The squares in the dataset span from 0 to 100 along the X and Y axes, so we'll only rasterize the portion from 0 to 50:

In [ ]:
raster = Raster.from_polygons('examples/polygons.geojson', field='my-data', bounds=[0,0,50,50,26911])
plot.raster(raster, title='Rasterized Polygons (Quadrant)', cmap='OrRd', clabel='Data Field')

## Raster.from_points

The syntax for the `from_points` factory is nearly identical to `from_polygons`, except that the file path should be for a Point/MultiPoint collection. When rasterizing points, raster pixels that contain a point will be set to `True` or the point's data field, as appropriate.

## Conclusion

In this tutorial, we've seen how to convert various types of datasets to `Raster` objects. We used:

* `from_file` to load a raster from the local file system
* `from_url` to load a raster from a web URL,
* `from_array` to add raster metadata to a numpy array
* `from_polygons` to rasterize a Polygon dataset, and
* `from_points` to rasterize a Point dataset.